<a href="https://colab.research.google.com/github/Nazhan99/Tensorflow_notes/blob/main/07_milestone_project_1_food_vision_big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project 1: Food Vision Big


## Check GPU
check gpu to ensure it compatible with mixed precision training. We nned to access to a Tesla T4 (from google colab) or if we are using our own hardware, our GPU needs a score of 7.0+ (refer: https://developer.nvidia.com/cuda-gpus)

to change Gpu, go to runtime>factory runtime type #this is randomly given

if not get access to 7.0+ GPU, the process can still run but slower.

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-a06d611b-213f-430e-8bef-abc131cf9251)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-02-27 04:49:43--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-02-27 04:49:43 (64.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
#import series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys